<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=58391766b93233fc84e5f61ae69304d3a4bdce7f894d88273027324ae2281f7d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 11:21:05
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -44.76 K (-0.31%)
Current PnL: -23.04 L (-15.1%)
CY Booked + Current PnL: -9.15 L (-5.99%)
-------------------
Total profit:  1.61 L
Total loss:  -24.65 L
-------------------
Total Booked + Current PnL: 17.83 L (14.19%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.73%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.45 L (61.96%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.03,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.09,-8.91,9.80,0.02,15583.0,-15548.0,159011.0,142.75,71.0,M-SC,5.45,234.0,-1.00,1.11,38.31,OX40N,NTT,PAINTS
83,VOLTAS,1530.00,-1.13,9.19,9.61,19.69,20120.0,17628.0,209370.0,-1.49,54.0,X-MC,3.57,78.0,0.88,1.47,16.62,XY25,NTT,AC
43,ITC,452.00,-0.32,-0.67,11.18,10.44,22226.0,-1335.0,198803.0,-40.74,47.0,X-LC,0.76,5.0,-0.06,1.39,4.86,X40,NTT,FMCG
35,ICICIGI,2252.93,0.32,7.27,12.05,20.19,22532.0,12666.0,186986.0,-16.02,52.0,X-MC,7.66,68.0,0.56,1.31,23.48,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.58,1.86,69.12,72.27,119303.0,3155.0,172603.0,-12.43,58.0,M-LC,4.59,99.0,0.03,1.21,8.15,XR,NTT,IT
7,ATULAUTO,844.00,1.46,-21.34,85.07,45.57,131907.0,-42075.0,155057.0,3354.05,42.0,M-SC,4.02,244.0,-0.32,1.09,9.93,XY24,NTT,AUTO
26,GILLETTE,11206.78,1.39,-0.85,33.55,32.42,84460.0,-2151.0,251745.0,-18.53,35.0,X-SC,6.65,85.0,-0.03,1.76,14.12,X40,ATH,FMCG
68,SONACOMS,806.63,1.38,-11.75,58.13,39.54,51891.0,-11891.0,89268.0,-31.03,69.0,M-SC,6.85,220.0,-0.23,0.63,25.88,AR,ATH,AUTO
50,LTIM,7201.88,1.32,9.76,19.95,31.66,53901.0,24029.0,270180.0,5.43,65.0,H-LC,12.55,43.0,0.45,1.89,49.91,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-3.43,-46.09,223.05,74.15,196721.0,-75409.0,88196.0,-258.45,25.0,H-SC,14.58,149.0,-0.38,0.62,13.05,XR,NTT,CHEMICALS
16,CERA,9475.00,-2.94,-29.12,74.80,23.89,93255.0,-51231.0,124672.0,-31.50,24.0,H-SC,3.99,157.0,-0.55,0.87,7.67,OX40N,NTT,CERAMICS
27,GLAXO,3466.20,-2.37,-2.10,42.13,39.15,82197.0,-4180.0,195104.0,-21.47,28.0,X-MC,9.07,60.0,-0.05,1.37,26.30,X40,ATH,PHARMA
6,ASIANTILES,137.00,-2.20,-14.68,112.93,81.67,90388.0,-13771.0,80039.0,7194.44,46.0,L-SC,18.49,271.0,-0.15,0.56,55.00,XR,NTT,CERAMICS
48,KPIGREEN,730.66,-2.17,-11.58,66.19,46.95,73333.0,-14505.0,110792.0,-18.10,27.0,H-SC,6.26,154.0,-0.20,0.78,38.80,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.69,-0.06,106.85,106.72,131705.0,-74.0,123262.0,-50.44,34.0,H-SC,4.1,139.0,-0.0,0.86,23.3,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.85,-4.80,118.39,107.91,166408.0,-7085.0,140559.0,-22.58,34.0,M-SC,10.35,216.0,-0.04,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.09,-8.91,9.80,0.02,15583.0,-15548.0,159011.0,142.75,71.0,M-SC,5.45,234.0,-1.00,1.11,38.31,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.03,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.47,-22.67,46.74,13.47,97467.0,-61137.0,208530.0,-68.12,26.0,H-SC,1.52,158.0,-0.63,1.46,6.03,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.54,-22.90,71.25,32.03,72418.0,-30195.0,101640.0,-16.54,49.0,H-SC,6.63,148.0,-0.42,0.71,13.85,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.58,1.86,69.12,72.27,119303.0,3155.0,172603.0,-12.43,58.0,M-LC,4.59,99.0,0.03,1.21,8.15,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.58,1.86,69.12,72.27,119303.0,3155.0,172603.0,-12.43,58.0,M-LC,4.59,99.0,0.03,1.21,8.15,XR,NTT,IT
38,INDIAMART,4810.62,-0.69,-0.06,106.85,106.72,131705.0,-74.0,123262.0,-50.44,34.0,H-SC,4.10,139.0,-0.00,0.86,23.30,AR,ATH,MISC
25,FINCABLES,1641.55,-0.85,-4.80,118.39,107.91,166408.0,-7085.0,140559.0,-22.58,34.0,M-SC,10.35,216.0,-0.04,0.98,0.00,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-0.36,-5.75,93.03,81.92,100651.0,-6606.0,108192.0,-44.81,31.0,M-SC,5.30,236.0,-0.07,0.76,28.08,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,-0.40,-2.53,39.15,35.63,79972.0,-5304.0,204270.0,-16.30,32.0,H-MC,4.76,119.0,-0.07,1.43,13.72,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JSWINFRA,342.00,-1.44,-10.38,28.57,15.23,51297.0,-20797.0,179550.0,-23.41,22.0,X-MC,6.53,66.0,-0.41,1.26,18.52,X40N,NTT,REALTY
76,TMPV,600.00,-1.44,-15.39,68.04,42.18,159124.0,-42542.0,233868.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.52,-35.13,108.43,35.20,90501.0,-45205.0,83465.0,9.46,25.0,X-SC,12.61,92.0,-0.50,0.58,0.52,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,0.45,-15.79,70.19,43.32,96273.0,-25712.0,137160.0,-27.69,26.0,X-SC,2.03,90.0,-0.27,0.96,5.31,X40N,ATH,ELECTRICAL
60,RELAXO,1176.00,-0.81,-47.53,190.26,52.31,144917.0,-68992.0,76168.0,-44.15,28.0,X-SC,3.78,91.0,-0.48,0.53,1.96,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-1.44,-15.39,68.04,42.18,159124.0,-42542.0,233868.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.32,-0.67,11.18,10.44,22226.0,-1335.0,198803.0,-40.74,47.0,X-LC,0.76,5.0,-0.06,1.39,4.86,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.15,1.13,29.51,30.97,52743.0,1992.0,178730.0,-22.64,48.0,X-MC,1.53,75.0,0.04,1.25,13.83,X40,ATH,INSURANCE
3,ACC,3906.0,-0.36,-23.36,114.19,64.15,208237.0,-55591.0,182360.0,-55.00,42.0,X-SC,1.90,82.0,-0.27,1.28,2.29,XY24,BTT,CEMENT
66,SIEMENS,4671.5,-0.21,-14.91,47.51,25.51,75230.0,-27750.0,158345.0,-2.87,53.0,H-LC,1.92,49.0,-0.37,1.11,16.69,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-1.44,-15.39,68.04,42.18,159124.0,-42542.0,233868.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.13,-13.57,71.76,48.45,124359.0,-27207.0,173299.0,-19.87,31.0,X-MC,7.47,64.0,-0.22,1.21,0.13,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,0.52,-35.13,108.43,35.20,90501.0,-45205.0,83465.0,9.46,25.0,X-SC,12.61,92.0,-0.50,0.58,0.52,X40,NTT,FOOTWEAR
56,QUESS,424.00,0.65,-28.84,100.75,42.84,46602.0,-18751.0,46255.0,-53.17,37.0,X-SC,37.26,83.0,-0.40,0.32,0.65,XY24,NTT,MISC
54,PAGEIND,51605.08,0.05,-4.96,32.64,26.07,50794.0,-8120.0,155620.0,-30.06,36.0,X-MC,8.86,55.0,-0.16,1.09,0.88,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.65,-28.84,100.75,42.84,46602.0,-18751.0,46255.0,-53.17,37.0,X-SC,37.26,83.0,-0.40,0.32,0.65,XY24,NTT,MISC
60,RELAXO,1176.00,-0.81,-47.53,190.26,52.31,144917.0,-68992.0,76168.0,-44.15,28.0,X-SC,3.78,91.0,-0.48,0.53,1.96,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.52,-35.13,108.43,35.20,90501.0,-45205.0,83465.0,9.46,25.0,X-SC,12.61,92.0,-0.50,0.58,0.52,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.11,2.55,20.99,24.08,27327.0,3239.0,130189.0,-5.81,44.0,X-SC,4.75,89.0,0.12,0.91,22.67,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.06,-4.91,27.87,21.58,37138.0,-6885.0,133254.0,-52.09,33.0,X-MC,7.02,56.0,-0.19,0.93,17.97,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.01,-14.39,39.32,19.28,116460.0,-49771.0,296185.0,-25.24,63.0,X-LC,7.64,1.0,-0.43,2.07,9.49,X40,ATH,IT
41,INFY,2275.00,1.27,6.51,45.40,54.87,151300.0,20372.0,333260.0,-17.09,63.0,X-LC,3.15,2.0,0.13,2.33,15.33,X40,BTT,IT
76,TMPV,600.00,-1.44,-15.39,68.04,42.18,159124.0,-42542.0,233868.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.03,-9.54,49.99,35.67,142819.0,-30146.0,285696.0,-20.30,41.0,X-LC,6.05,4.0,-0.21,2.00,2.99,X40N,ATH,FMCG
43,ITC,452.00,-0.32,-0.67,11.18,10.44,22226.0,-1335.0,198803.0,-40.74,47.0,X-LC,0.76,5.0,-0.06,1.39,4.86,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.20,-14.68,112.93,81.67,90388.0,-13771.0,80039.0,7194.44,46.0,L-SC,18.49,271.0,-0.15,0.56,55.00,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.38,-11.75,58.13,39.54,51891.0,-11891.0,89268.0,-31.03,69.0,M-SC,6.85,220.0,-0.23,0.63,25.88,AR,ATH,AUTO
51,MASFIN,398.61,-2.03,-4.90,28.34,22.05,26407.0,-4800.0,93180.0,-17.80,51.0,H-SC,7.22,164.0,-0.18,0.65,35.44,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.74,-20.26,54.29,23.03,77526.0,-36271.0,142800.0,55.70,47.0,H-SC,11.07,167.0,-0.47,1.00,41.69,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.09,-8.91,9.80,0.02,15583.0,-15548.0,159011.0,142.75,71.0,M-SC,5.45,234.0,-1.00,1.11,38.31,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.09,-8.91,9.80,0.02,15583.0,-15548.0,159011.0,142.75,71.0,M-SC,5.45,234.0,-1.00,1.11,38.31,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.09,0.45,20.40,20.95,51592.0,1135.0,252903.0,-0.49,76.0,X-LC,12.73,31.0,0.02,1.77,36.25,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.55,-37.16,111.47,32.89,54083.0,-28688.0,48518.0,-699.18,65.0,L-MC,5.63,259.0,-0.53,0.34,33.62,XR,NTT,BANKS
50,LTIM,7201.88,1.32,9.76,19.95,31.66,53901.0,24029.0,270180.0,5.43,65.0,H-LC,12.55,43.0,0.45,1.89,49.91,X200,ATH,IT
0,5PAISA,593.00,-1.97,-36.01,76.44,12.90,106877.0,-78678.0,139818.0,105.53,56.0,H-SC,14.56,161.0,-0.74,0.98,16.26,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.80
1,25,44.69
2,50,75.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.50
MC    29.28
LC    25.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.87
X40      21.23
X40N     11.83
XR        9.63
XY25      9.13
AR        8.96
OX40N     7.66
X200      1.89
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.89
X-MC    22.87
X-LC    20.01
M-SC    12.06
X-SC     8.16
H-MC     4.70
H-LC     3.00
L-SC     1.39
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.80,-4.91,38.83
IT,13.85,-14.18,73.13
FINANCE,9.41,-15.19,63.86
MISC,7.41,-22.60,72.82
PAINTS,5.99,-7.81,23.82
ELECTRICAL,5.89,-11.70,52.61
INSURANCE,4.40,-0.71,36.20
PHARMA,3.84,-2.10,34.77
AUTO,3.36,-20.18,71.71


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3057220.0,21
AR,1292485.0,10
XR,1289234.0,13
X40,1006674.0,14
X40N,757162.0,9
OX40N,696892.0,10
XY25,338612.0,6
SR,279267.0,2
MH,73333.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3567900.0,25
M-SC,1338405.0,15
X-MC,1217293.0,16
X-LC,905111.0,11
X-SC,752520.0,8
H-MC,407960.0,3
L-SC,262771.0,3
H-LC,129131.0,2
M-LC,119303.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1221734.0      6
           AR         877151.0      5
           XR         792101.0      7
M-SC       XY24       782766.0      6
X-MC       X40        443671.0      7
X-LC       X40        388042.0      5
X-MC       XY24       348909.0      3
H-SC       OX40N      324314.0      4
X-SC       X40N       295393.0      3
M-SC       OX40N      284195.0      5
X-SC       XY24       282166.0      3
H-SC       SR         279267.0      2
X-MC       X40N       249355.0      4
X-LC       XY24       231704.0      2
H-MC       AR         218019.0      2
X-LC       X40N       212414.0      2
H-MC       XY24       189941.0      1
X-MC       XY25       175358.0      2
X-SC       X40        174961.0      2
L-SC       XR         174388.0      2
M-SC       XR         149359.0      2
           AR         122085.0      2
M-LC       XR         119303.0      1
L-SC       OX40N       88383.0      1
H-LC       AR          75230.0      1
H-SC       MH          73333.0      1
X-LC       XY25        72951.0      2
L-MC       XR          54083.0      1
H-LC       X200        53901.0      1
M-MC       XY25        49716.0      1
L-LC       XY25        40587.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
